# Metric Analysis for ECON model

In [13]:
%reload_ext autoreload
%autoreload 2

# from common.metrics.gradient import Gradient
import torch
import torchinfo
import pytorch_lightning as pl 
import os
import sys

# import modules from ECON model
module_path = os.path.abspath(os.path.join('../../workspace/models/econ/code/')) # or the path to your source code
sys.path.insert(0, module_path)
from q_autoencoder import AutoEncoder
from autoencoder_datamodule import AutoEncoderDataModule

# import modules from common metrics
module_path = os.path.abspath(os.path.join('../../workspace/common/metrics/')) # or the path to your source code
sys.path.insert(0, module_path)
from gradient import Gradient



Select the desired ECON model:

In [14]:
base_path = "/loss_landscape/checkpoint/different_knobs_subset_10/"
batch_size = 1024
learning_rate = 0.1
precision = 8
size = 'small'

In [15]:
def load_model(path, batch_size, learning_rate, precision, size):
    model_path = path + f'bs{batch_size}_lr{learning_rate}/ECON_{precision}b/{size}/net_1_best.pkl'
    model = AutoEncoder(
        quantize=(precision < 32),
        precision=[
            precision,
            precision,
            precision+3
        ],
        learning_rate=learning_rate,
        econ_type=size
    )
    model(torch.randn((1, 1, 8, 8)))  # Update tensor shapes 
    print(model_path)
    model_param = torch.load(model_path)
    model.load_state_dict(model_param['state_dict'])
    return model


model = load_model(base_path, batch_size, learning_rate, precision, size)
torchinfo.summary(model, input_size=(1, 1, 8, 8))  # (B, C, H, W)            

/loss_landscape/checkpoint/different_knobs_subset_10/bs1024_lr0.1/ECON_8b/small/net_1_best.pkl


Layer (type:depth-idx)                   Output Shape              Param #
AutoEncoder                              [1, 1, 8, 8]              --
├─QuantizedEncoder: 1-1                  [1, 16]                   --
│    └─QuantAct: 2-1                     [1, 1, 8, 8]              --
│    └─QuantConv2d: 2-2                  [1, 1, 4, 4]              20
│    └─ReLU: 2-3                         [1, 1, 4, 4]              --
│    └─QuantAct: 2-4                     [1, 1, 4, 4]              --
│    └─Flatten: 2-5                      [1, 16]                   --
│    └─QuantLinear: 2-6                  [1, 16]                   272
│    └─ReLU: 2-7                         [1, 16]                   --
├─Sequential: 1-2                        [1, 1, 8, 8]              --
│    └─Linear: 2-8                       [1, 128]                  2,176
│    └─ReLU: 2-9                         [1, 128]                  --
│    └─Unflatten: 2-10                   [1, 8, 4, 4]              --
│    └─Conv

Setup the data loader:

In [16]:
data_path = '../../data/ECON/Elegun'
processed_file = 'nELinks5.npy'

def get_data_module(batch_size):
    data_module = AutoEncoderDataModule(
        data_dir=data_path,
        data_file=os.path.join(data_path, processed_file),
        batch_size=batch_size,
        num_workers=4
        )
    # checek if we have processed the data
    if not os.path.exists(os.path.join(data_path, processed_file)):
        print('Processing the data...')
        data_module.process_data(save=True)

    data_module.setup(0)
    return data_module

data_module = get_data_module(batch_size)

Loaded shaped data shape: (1740950, 1, 8, 8)
Loaded shaped data datatype: float32


Model performances:

In [20]:
trainer = pl.Trainer(
        max_epochs=5,
        accelerator="auto",
        devices=1
    )

def compute_EMD(size, precision, bs, lr):
    # load the model 
    model = load_model(base_path, bs, lr, precision, size)
    # get the data loader
    data_module = get_data_module(bs)
    _, val_sum = data_module.get_val_max_and_sum()
    model.set_val_sum(val_sum)
    data_module.setup("test")
    avg_emd = trainer.test(model, dataloaders=data_module.test_dataloader(), verbose=False)
    print(f'BATCH SIZE: {bs} - LEARNING RATE {lr} - SIZE {size} - PRECISION {precision}')
    print("AVG EMD: ", avg_emd)
    

compute_EMD('small', 8, batch_size, learning_rate)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


/loss_landscape/checkpoint/different_knobs_subset_10/bs1024_lr0.1/ECON_8b/small/net_1_best.pkl
Loaded shaped data shape: (1740950, 1, 8, 8)
Loaded shaped data datatype: float32
Reading files ['5Elinks_data.csv', 'nELinks5.npy']
Input data shape: (1740950, 48)


You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Prepped shaped data shape: (1740950, 1, 8, 8)
Loaded shaped data shape: (1740950, 1, 8, 8)
Loaded shaped data datatype: float32
Testing DataLoader 0: 100%|██████████| 341/341 [02:34<00:00,  2.21it/s]
BATCH SIZE: 1024 - LEARNING RATE 0.1 - SIZE small - PRECISION 8
AVG EMD:  [{'AVG_EMD': 13.577227592468262}]


## Gradient computation

In [18]:
_, val_loader = data_module.dataloaders()

metric = Gradient(
    model=model,
    data_loader=val_loader,
    loss=model.loss
)

results = metric.compute()
results

Computing the gradients...


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.